<a href="https://colab.research.google.com/github/skkzsh/llm-notebook/blob/main/langchain_summarize_epub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# これは何？
LangChainに本 (EPub) を要約してもらいます.

# 準備

まずモジュールをインストールします.

In [ ]:
!pip install openai tiktoken langchain unstructured

# EPubファイルを用意
ココでは例として [process_book.epub](https://github.com/Shinpeim/process-book/tree/master/release) (CCライセンス) を要約してもらいます.  
Google Driveにダウンロードしてください.  

Google DriveをColabにmountします.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

EPubファイルを配置したパスを変数として定義しておきます.

In [ ]:
epub = '/content/drive/MyDrive/process_book.epub'

# EPubファイルをロード
[`UnstructuredEPubLoader`](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/epub.html) で, EPubファイルをロードできます.

In [ ]:
%%time
from langchain.document_loaders import UnstructuredEPubLoader

loader = UnstructuredEPubLoader(epub)
docs = loader.load_and_split()

ロードした内容は分割されています.

In [ ]:
len(docs)

ロードした内容を出力すると, EPubファイルの文章が格納されています.  
分割した文章は, 重複部分があることも分かります (`chunk_overlap`).

In [ ]:
docs

# 要約させる
`OPENAI_API_KEY` を設定します.

In [ ]:
%env OPENAI_API_KEY=

モデルには gpt-3.5-turbo を使います.

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

[`load_summarize_chain`](https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html) で要約ができます.
- `chain_type` は `map_reduce` を指定してます.  
- 処理経過も確認するために `verbose` を `True` にしておきます (以降も同様).

In [ ]:
from langchain.chains.summarize import load_summarize_chain

summary_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    verbose=True,
)

要約を実行します.

In [ ]:
%%time
summary = summary_chain.run(docs)

要約結果を出力します.

In [ ]:
summary

しかし, 要約結果が英語で返ってきてしまいました.  
それに, 要約結果も簡潔すぎるかもしれません.

# 要約を日本語で得る
要約実行時のログを見ると, デフォルトのプロンプトが英語 (「Write a concise summary of the following:」など) のため, 要約結果も英語になってしまったようです.  
そこで, 日本語のプロンプトに上書きします.  
文字数指定は適宜調整してください.

In [ ]:
from langchain import PromptTemplate

map_template = """以下は書籍の内容の一部です。2000文字以内で要約してください。:

########
{text}
########

要約:"""

combine_template = """1つの書籍を分割して、分割した内容に対して、それぞれ要約した結果を順に以下に並べています。この書籍を1000~2000文字ぐらいで要約してください。:

########
{text}
########

書籍の要約:"""

map_prompt = PromptTemplate(
    template=map_template,
    input_variables=["text"],
)

combine_prompt = PromptTemplate(
    template=combine_template,
    input_variables=["text"],
)

ja_summary_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    # return_intermediate_steps=True,
    verbose=True,
)

要約を実行します.

In [ ]:
%%time
ja_summary = ja_summary_chain.run(docs)

要約結果を出力します.

In [ ]:
ja_summary

要約結果を日本語で, より詳細に得ることができました.

# ライセンス
[![クリエイティブ・コモンズ・ライセンス](http://i.creativecommons.org/l/by-sa/3.0/88x31.png)](http://creativecommons.org/licenses/by-sa/3.0/deed.ja)
このノートは[クリエイティブ・コモンズ 表示 - 継承 3.0 非移植 ライセンス](http://creativecommons.org/licenses/by-sa/3.0/deed.ja)の下で公開します.